In [1]:
import pickle
import math
import pandas as pd
import os
from coir.data_loader import get_tasks, load_data_from_hf
from IPython.display import display, HTML
import ipywidgets as widgets
import html


In [2]:
cosqa_deepseek_path='/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/CS696DS-Oracle-Retrieving-Code-Explanations/Explanation_Generation/Cosqa/postprocessing/output/COSQA_deepseek_explanations_clean.csv'
cosqa_granite_path = '/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/CS696DS-Oracle-Retrieving-Code-Explanations/Explanation_Generation/Cosqa/postprocessing/output/COSQA_granite_explanations_clean.csv'

tasks = get_tasks(tasks=["cosqa"])
corpus, queries, qrels = tasks['cosqa']
deepseek_df = pd.read_csv(cosqa_deepseek_path)
granite_df = pd.read_csv(cosqa_granite_path)

in tasks 
fetched data from hf
data loader init


0it [00:00, ?it/s]

In [3]:
merged_df = pd.merge(
    deepseek_df,
    granite_df,
    on=["query_id", "corpus_id"],
    how="outer",
    suffixes=("_deepseek", "_granite")
)
# merged_df.columns

In [4]:
def mark_good_queries(df, k=10):
    """
    Given a retrieval DataFrame with columns:
      [query_id, retrieved_doc_id, score, ground_truth_relevance],
    returns a DataFrame with columns [query_id, is_good]
    where is_good is True if there's at least 1 relevant doc in top K.
    """
    df_sorted = df.sort_values(["query_id", "score"], ascending=[True, False])
    
    def top_k_has_relevant(subdf):
        topk = subdf.head(k)
        return int((topk["ground_truth_relevance"] == 1).any())

    query_good = df_sorted.groupby("query_id").apply(top_k_has_relevant).reset_index()
    query_good.columns = ["query_id", "is_good"]
    
    return query_good

def first_relevant_rank(df):
    df_sorted = df.sort_values(["query_id", "score"], ascending=[True, False])
    def get_rank(subdf):
        subdf = subdf.reset_index(drop=True)
        rel = subdf[subdf["ground_truth_relevance"] == 1]
        return rel.index[0] + 1 if not rel.empty else None
    rank_df = df_sorted.groupby("query_id").apply(get_rank).reset_index()
    rank_df.columns = ["query_id", "first_rel_rank"]
    return rank_df


Comparison of correct queries between methods

In [5]:
# LOAD RESULTS FOR METHOD 1
dataset_1 = "cosqa_noComments"
method_1 = "baseline"
retrieval_1 = "dres"
encoder_1 = "intfloat/e5-base-v2"

if retrieval_1 == "bm25":
    results_path = f"/work/pi_wenlongzhao_umass_edu/27/vaishnavisha/CS696DS-Oracle-Retrieving-Code-Explanations/coir-main/results/{dataset_1}/{method_1}/{retrieval_1}/retrieval_evaluation.csv"
else:
    results_path = f"/work/pi_wenlongzhao_umass_edu/27/vaishnavisha/CS696DS-Oracle-Retrieving-Code-Explanations/coir-main/results/{dataset_1}/{method_1}/{retrieval_1}/{encoder_1}/retrieval_evaluation.csv"

first_df = pd.read_csv(results_path)

In [6]:
# LOAD RESULTS FOR METHOD 2
dataset_2 = "cosqa_noComments"
method_2 = "granite1"
retrieval_2 = "dres"
encoder_2 = "intfloat/e5-base-v2"

if retrieval_2 == "bm25":
    results_path = f"/work/pi_wenlongzhao_umass_edu/27/vaishnavisha/CS696DS-Oracle-Retrieving-Code-Explanations/coir-main/results/{dataset_2}/{method_2}/{retrieval_2}/retrieval_evaluation.csv"
else:
    results_path = f"/work/pi_wenlongzhao_umass_edu/27/vaishnavisha/CS696DS-Oracle-Retrieving-Code-Explanations/coir-main/results/{dataset_2}/{method_2}/{retrieval_2}/{encoder_2}/retrieval_evaluation.csv"
second_df = pd.read_csv(results_path)

In [7]:
k = 10 # Hyper-param to select top k documents to classify a "good" retrieval
first_good = mark_good_queries(first_df, k=k)
second_good = mark_good_queries(second_df, k=k)

first_ranks = first_relevant_rank(first_df)
second_ranks = first_relevant_rank(second_df)

compare_df = first_good.merge(second_good, on="query_id", suffixes=(f"_{method_1}", f"_{method_2}"))
compare_df = compare_df.merge(first_ranks, on="query_id")
compare_df = compare_df.merge(second_ranks, on="query_id", suffixes=(f"_{method_1}", f"_{method_2}"))

compare_df.head()


,query_id,is_good_baseline,is_good_granite1,first_rel_rank_baseline,first_rel_rank_granite1
0,q20105,1,1,1.0,1.0
1,q20106,1,1,1.0,1.0
2,q20107,1,0,6.0,88.0
3,q20108,1,1,1.0,9.0
4,q20109,1,1,1.0,1.0


In [8]:
if "codesearchnet" in dataset_1 or "codesearchnet" in dataset_2:
    tasks = load_data_from_hf("CodeSearchNet-python")
    corpus, queries, qrels = tasks
compare_df["query_text"] = compare_df["query_id"].map(queries)

bad_method_1 = compare_df[
    (compare_df[f"is_good_{method_1}"] == 1) &
    (compare_df[f"is_good_{method_2}"] == 0)
]
print(f"Queries where {method_1} is good, but {method_2} is bad:")
print(bad_method_1)

bad_method_2 = compare_df[
    (compare_df[f"is_good_{method_1}"] == 0) &
    (compare_df[f"is_good_{method_2}"] == 1)
]
print(f"Queries where {method_2} is good, but {method_1} is bad:")
print(bad_method_2)

# Queries both missed
both_missed = compare_df[
    (compare_df[f"is_good_{method_1}"] == 0) &
    (compare_df[f"is_good_{method_2}"] == 0)
]
print("Queries both missed:")
print(both_missed)

# Queries both got
both_got = compare_df[
    (compare_df[f"is_good_{method_1}"] == 1) &
    (compare_df[f"is_good_{method_2}"] == 1)
]
print("Queries both got:")
print(both_got)


Queries where baseline is good, but granite1 is bad:
    query_id  is_good_baseline  is_good_granite1  first_rel_rank_baseline  \
2     q20107                 1                 0                      6.0   
12    q20117                 1                 0                      1.0   
42    q20147                 1                 0                      6.0   
43    q20148                 1                 0                      1.0   
55    q20160                 1                 0                      1.0   
63    q20168                 1                 0                      1.0   
91    q20196                 1                 0                     10.0   
127   q20232                 1                 0                      1.0   
149   q20254                 1                 0                      1.0   
161   q20266                 1                 0                      9.0   
163   q20268                 1                 0                     10.0   
181   q20286           

In [9]:
good_method_1_count = compare_df[f"is_good_{method_1}"].sum()
good_method_2_count = compare_df[f"is_good_{method_2}"].sum()
bad_method_1_count = len(compare_df) - good_method_1_count
bad_method_2_count = len(compare_df) - good_method_2_count

print("Total queries:", len(compare_df))
print(f"{method_1.capitalize()} good queries:", good_method_1_count)
print(f"{method_1.capitalize()} bad queries:", bad_method_1_count)
print(f"{method_2.capitalize()} good queries:", good_method_2_count)
print(f"{method_2.capitalize()} bad queries:", bad_method_2_count)


# Queries where method_1 is good but method_2 is bad
bad_method_2_subset = compare_df[
    (compare_df[f"is_good_{method_1}"] == 1) &
    (compare_df[f"is_good_{method_2}"] == 0)
]
avg_rank_bad_method_2 = bad_method_2_subset[f"first_rel_rank_{method_2}"].mean()
print(f"\n{method_1.capitalize()} good but {method_2.capitalize()} bad queries count:", len(bad_method_2_subset))
#print(f"Average first relevant doc rank ({method_2.capitalize()}) for these queries:", avg_rank_bad_method_2)

# Queries where method_2 is good but method_1 is bad
bad_method_1_subset = compare_df[
    (compare_df[f"is_good_{method_1}"] == 0) &
    (compare_df[f"is_good_{method_2}"] == 1)
]
avg_rank_bad_method_1 = bad_method_1_subset[f"first_rel_rank_{method_1}"].mean()
print(f"\n{method_2.capitalize()} good but {method_1.capitalize()} bad queries count:", len(bad_method_1_subset))
#print(f"Average first relevant doc rank ({method_1.capitalize()}) for these queries:", avg_rank_bad_method_1)

# Queries both missed
print(f"\nQueries both missed: {len(both_missed)}")

# Queries both got
avg_rank_method_1_both = both_got[f"first_rel_rank_{method_1}"].mean()
avg_rank_method_2_both = both_got[f"first_rel_rank_{method_2}"].mean()
print(f"\nQueries both got: {len(both_got)}")
#print(f"Average first relevant doc rank ({method_1.capitalize()}) for these queries:", avg_rank_method_1_both)
#print(f"Average first relevant doc rank ({method_2.capitalize()}) for these queries:", avg_rank_method_2_both)

Total queries: 500
Baseline good queries: 305
Baseline bad queries: 195
Granite1 good queries: 344
Granite1 bad queries: 156

Baseline good but Granite1 bad queries count: 37

Granite1 good but Baseline bad queries count: 76

Queries both missed: 119

Queries both got: 268


In [10]:
def make_group_table(df, title):
    display_df = df[["query_id", "query_text"]].copy()
    html = f"<h3>{title} (n={len(display_df)})</h3>"
    html += display_df.to_html(index=False, escape=False)
    return html

html_out = ""
html_out += make_group_table(bad_method_1, f"Queries where {method_1} is good, but {method_2} is bad")
html_out += make_group_table(bad_method_2, f"Queries where {method_2} is good, but {method_1} is bad")
html_out += make_group_table(both_missed, "Queries both missed")
html_out += make_group_table(both_got, "Queries both got")

#display(HTML(html_out))

In [11]:
if "codesearchnet" in dataset_1 or "codesearchnet" in dataset_2:
    csn_expl_path = '/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/explanations/CodeSearchNet_Python_deepseek_allSplits.csv'
    merged_df = pd.read_csv(csn_expl_path)
    bad_method_1_merge = pd.merge(bad_method_1, merged_df, on="query_id", how="left")
    bad_method_2_merge = pd.merge(bad_method_2, merged_df, on="query_id", how="left")
    both_missed_merge = pd.merge(both_missed, merged_df, on="query_id", how="left")
    both_got_merge = pd.merge(both_got, merged_df, on="query_id", how="left")
else:
    bad_method_1_merge = pd.merge(bad_method_1, merged_df, on="query_id", how="left")
    bad_method_2_merge = pd.merge(bad_method_2, merged_df, on="query_id", how="left")
    both_missed_merge = pd.merge(both_missed, merged_df, on="query_id", how="left")
    both_got_merge = pd.merge(both_got, merged_df, on="query_id", how="left")

In [12]:
import ipywidgets as widgets
from IPython.display import display, HTML

# Available columns for explanations and code.
if "codesearchnet" in dataset_1 or "codesearchnet" in dataset_2:
    available_explanation_columns = [
    "explanation_deepseek_1_cleaned", "explanation_deepseek_2_cleaned", 
    "explanation_deepseek_3_cleaned", "explanation_deepseek_4_cleaned", 
    "explanation_deepseek_5_cleaned"
]

    available_code_columns = [
        "code",
        "explanation_deepseek_1_cleaned", "explanation_deepseek_2_cleaned", 
        "explanation_deepseek_3_cleaned", "explanation_deepseek_4_cleaned", 
        "explanation_deepseek_5_cleaned"
    ]

else:
    available_explanation_columns = [
        "explanation_deepseek_1", "explanation_deepseek_2", "explanation_deepseek_3", 
        "explanation_deepseek_4", "explanation_deepseek_5", 
        "explanation_deepseek_1_cleaned", "explanation_deepseek_2_cleaned", 
        "explanation_deepseek_3_cleaned", "explanation_deepseek_4_cleaned", 
        "explanation_deepseek_5_cleaned", 
        "explanation_granite_1", "explanation_granite_2", "explanation_granite_3", 
        "explanation_granite_4", "explanation_granite_5", 
        "explanation_granite_1_cleaned", "explanation_granite_2_cleaned", 
        "explanation_granite_3_cleaned", "explanation_granite_4_cleaned", 
        "explanation_granite_5_cleaned"
    ]

    available_code_columns = [
        "code_deepseek", "cleaned_code_deepseek",
        "explanation_deepseek_1", "explanation_deepseek_2", "explanation_deepseek_3", 
        "explanation_deepseek_4", "explanation_deepseek_5", 
        "explanation_deepseek_1_cleaned", "explanation_deepseek_2_cleaned", 
        "explanation_deepseek_3_cleaned", "explanation_deepseek_4_cleaned", 
        "explanation_deepseek_5_cleaned", 
        "explanation_granite_1", "explanation_granite_2", "explanation_granite_3", 
        "explanation_granite_4", "explanation_granite_5", 
        "explanation_granite_1_cleaned", "explanation_granite_2_cleaned", 
        "explanation_granite_3_cleaned", "explanation_granite_4_cleaned", 
        "explanation_granite_5_cleaned"
    ]

def make_html_block(text):
    """Wrap the given text in a <div> that preserves whitespace and left-aligns the text."""
    return f"<div style='white-space: pre-wrap; text-align: left; font-family: monospace;'>{text}</div>"

def show_page(df, code_col, expl_col, page=1, page_size=5):
    """
    Display a subset of rows from the DataFrame using the selected
    code column (for code) and explanation column (for explanation).
    The resulting table will have headers that exactly match the dropdown selections.
    """
    # Select the desired columns; keep original names.
    display_df = df[["query_id", "query_text", code_col, expl_col]].copy()
    
    # Apply HTML formatting to the code column.
    display_df[code_col] = display_df[code_col].apply(make_html_block)
    
    # Paginate.
    start = (page - 1) * page_size
    end = start + page_size
    sub_df = display_df.iloc[start:end]
    
    # Use the Styler to render HTML.
    styled = sub_df.style.format({code_col: lambda s: s})
    html = styled.to_html()
    display(HTML(html))

# Example interactive calls:
# (Replace the DataFrame variables below with your actual DataFrames,
# e.g. bad_method_1_merge, bad_method_2_merge, both_missed_merge, both_got_merge.)
print(f"Group: Queries where {method_1} is good, but {method_2} is bad")
widgets.interact(lambda page, code, expl: show_page(bad_method_1_merge, code, expl, page=page),
                 page=widgets.IntSlider(min=1, max=(len(bad_method_2_subset) // 5) + 1, step=1, value=1),
                 code=widgets.Dropdown(options=available_code_columns, value=available_code_columns[0], description="Code"),
                 expl=widgets.Dropdown(options=available_explanation_columns, value=available_explanation_columns[0], description="Explanation"))

print(f"Group: Queries where {method_2} is good, but {method_1} is bad")
widgets.interact(lambda page, code, expl: show_page(bad_method_2_merge, code, expl, page=page),
                 page=widgets.IntSlider(min=1, max=(len(bad_method_1_subset) // 5) + 1, step=1, value=1),
                 code=widgets.Dropdown(options=available_code_columns, value=available_code_columns[0], description="Code"),
                 expl=widgets.Dropdown(options=available_explanation_columns, value=available_explanation_columns[0], description="Explanation"))

print("Group: Queries both missed")
widgets.interact(lambda page, code, expl: show_page(both_missed_merge, code, expl, page=page),
                 page=widgets.IntSlider(min=1, max=(len(both_missed) // 5) + 1, step=1, value=1),
                 code=widgets.Dropdown(options=available_code_columns, value=available_code_columns[0], description="Code"),
                 expl=widgets.Dropdown(options=available_explanation_columns, value=available_explanation_columns[0], description="Explanation"))

print("Group: Queries both got")
widgets.interact(lambda page, code, expl: show_page(both_got_merge, code, expl, page=page),
                 page=widgets.IntSlider(min=1, max=(len(both_got) // 5) + 1, step=1, value=1),
                 code=widgets.Dropdown(options=available_code_columns, value=available_code_columns[0], description="Code"),
                 expl=widgets.Dropdown(options=available_explanation_columns, value=available_explanation_columns[0], description="Explanation"))


Group: Queries where baseline is good, but granite1 is bad


interactive(children=(IntSlider(value=1, description='page', max=8, min=1), Dropdown(description='Code', optio…

Group: Queries where granite1 is good, but baseline is bad


interactive(children=(IntSlider(value=1, description='page', max=16, min=1), Dropdown(description='Code', opti…

Group: Queries both missed


interactive(children=(IntSlider(value=1, description='page', max=24, min=1), Dropdown(description='Code', opti…

Group: Queries both got


interactive(children=(IntSlider(value=1, description='page', max=54, min=1), Dropdown(description='Code', opti…

<function __main__.<lambda>(page, code, expl)>